In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **Lightning 8 Step (기본 4 step)**
Lightning8Step = False #@param {type:"boolean"}

#@markdown **한글 번역**
Ko_Language = False #@param {type:"boolean"}

#@markdown -----
#@markdown **구글드라이브 연동폴더**
#@markdown * fooocus/outputs : 생성된 이미지들이 저장되요
#@markdown * fooocus/loras : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * fooocus/embeddings : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요



#@markdown **체크포인트 URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/149660?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **체크포인트 파일명**
Checkpoint_Filename = 'cherry_picker_xl.safetensors' #@param {type:"string"}

Preset = ''
NoteBookName = 'cherry_picker_xl_webui_colab'

Workspace = 'fooocus'
GoogleHomePath = f'/content/drive/MyDrive/{Workspace}'
HomePath = '/content/Fooocus'

PRESET_PARAM = ""
LANGUAGE_PARAM = ""

import os

if not Checkpoint_Filename:
  if Checkpoint_Url:
    Checkpoint_Filename = "checkpoint.safetensors"
    
if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

!pip install pygit2==1.12.2
%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git {HomePath}
  
if Lightning8Step:
  !sed -i -e "s/LIGHTNING = 4/LIGHTNING = 8/" {HomePath}/modules/flags.py
  !sed -i -e "s/sdxl_lightning_4step_lora/sdxl_lightning_8step_lora/" {HomePath}/modules/config.py
    
if Google_Drive:
  !mkdir {GoogleHomePath}
  !mkdir {GoogleHomePath}/loras
  !mkdir {GoogleHomePath}/upscale_models
  !mkdir {GoogleHomePath}/embeddings
  !mkdir {GoogleHomePath}/outputs
  !wget https://raw.githubusercontent.com/ninjaneural/fooocus/master/misc/config.txt -O {HomePath}/config.txt

if Ko_Language:
  !wget https://raw.githubusercontent.com/ninjaneural/fooocus/master/misc/ko.json -O {HomePath}/language/ko.json
  LANGUAGE_PARAM = '--language ko'

if Preset:
  if Checkpoint_Url:
    PRESET_PARAM = f'--preset colab'
    !wget https://raw.githubusercontent.com/ninjaneural/fooocus/master/misc/{Preset}.json -O {HomePath}/presets/colab.json
    !sed -i -e "s/#Checkpoint_Url#/{Checkpoint_Url}/" {HomePath}/presets/colab.json
    !sed -i -e "s/#Checkpoint_Filename#/{Checkpoint_Filename}/" {HomePath}/presets/colab.json
  else:
    PRESET_PARAM = f'--preset {Preset}'
else:
  PRESET_PARAM = f'--preset colab'
  !wget https://raw.githubusercontent.com/ninjaneural/fooocus/master/misc/presets/{NoteBookName}.json -O {HomePath}/presets/colab.json


%cd {HomePath}
!python entry_with_update.py --share {LANGUAGE_PARAM} {PRESET_PARAM} --port 18888

